In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
import statistics
import seaborn as sns
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
import re 


/home/larissa/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    #if arguments_list["comparison_list"] is None:
    #    arguments_list["comparison_list"] = np.nan

    if arguments_list["meanchange"] is None:
        arguments_list["meanchange"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list ["num_cell_filter"] is None:
        arguments_list["num_cell_filter"] = 0

    if arguments_list["reg"] is None:
        raise ValueError("Please provide a regulon csv.")

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    #is the naming of tf fine like this?
    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight" in arguments_list["reg"]:
        raise NameException("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns 'source', 'target' and 'weight'!")
    
    if arguments_list["plot"] is None:
        arguments_list["plot"] = True
    elif not isinstance(arguments_list["plot"], (bool)):
        raise ValueError("lot argument must be a boolean value.")
        
   
    return(arguments_list)



In [4]:
class TFObj:
    def __init__(self,
    tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        self.tf_activities_condition = tf_activities_condition
        self.tf_activities_cluster = tf_activities_cluster
        self.average_gene_expression = average_gene_expression
        self.regulon = regulon
        self.CTR_input_condition = CTR_input_condition
        self.CTR_input_cluster = CTR_input_cluster
        self.intracellular_network_condition = intracellular_network_condition
        self.intracellular_network_cluster = intracellular_network_cluster

def make_TFOBj(tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        tf = TFObj(tf_activities_condition,
            tf_activities_cluster,
            average_gene_expression,
            regulon,
            CTR_input_condition,
            CTR_input_cluster,
            intracellular_network_condition,
            intracellular_network_cluster)

        return tf

In [5]:
def AverageExpression(sub_object, celltype = None, name_iterable = None, outpath = None):
    gene_ids = sub_object.var.index.values
    obs = sub_object[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    avg_df = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs[celltype])
    avg_df = avg_df.groupby(level=0, observed=False).mean()
    avg_df.T.to_csv(outpath + name_iterable + "_average_gene_expression_by_cluster_exp.csv")

    return avg_df.T


In [6]:
def eval_pval(p_val):
    p_val = float(p_val)
    if p_val < 0.001: 
      txt = "***"
    elif p_val < 0.01: 
      txt = "**"
    elif p_val < 0.05: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)


def eval_meanchange_tag(meanchange):
    if meanchange >= 1.0: 
      txt = "***"
    elif meanchange > 0.5: 
      txt = "**"
    elif meanchange > 0.0: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)

In [7]:
def create_unfiltered_tf_scores(tf_scores_df, condition, celltype, out_path):   
    summarized_tf_scores_df = tf_scores_df.groupby(celltype, observed = True).mean().T
    #tf_scores_df.groupby(celltype, observed = True).apply(display)
    #agg(["mean", "var"])
    summarized_tf_scores_df.to_csv(out_path + "/unfiltered_tf_scores_" + condition + ".csv")
    return summarized_tf_scores_df

#np.var() returns a value that is different from R's var(), whereas statistics.variance() is the same as R's var()
def save_variable_tf_score(filtered_summarized_tf_scores_df, condition, out_path, plot):
    filtered_summarized_tf_scores_df["var"] = filtered_summarized_tf_scores_df.apply(statistics.variance, axis=1).unique()
    filtered_summarized_tf_scores_df.to_csv(out_path + "/variable_tf_scores_" + condition + ".csv")

    if plot:
        top_variable_tfs = filtered_summarized_tf_scores_df.sort_values("var", ascending=False).head(n=20).drop(columns="var")
        cluster_map = sns.clustermap(top_variable_tfs, cmap="vlag", center=0, vmin=top_variable_tfs.min(axis=None), cbar_kws={"label": "t-score"},
                                    cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
        cluster_map.ax_heatmap.set_xlabel("Cell Type")
        cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=45) 
        plt.savefig(out_path + "/tf_activity_top20_variable_" + condition + ".pdf", bbox_inches='tight')
        plt.close()

    filtered_summarized_tf_scores_df_var = filtered_summarized_tf_scores_df
    return filtered_summarized_tf_scores_df_var
    

In [8]:
def plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping, condition, out_path):
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.drop(columns="var")
    calc_size = ((len(filtered_summarized_tf_scores_df.columns.unique()) * 1.5), (len(filtered_summarized_tf_scores_df) * 0.06))
    cluster_map = sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "t-score"}, figsize=calc_size, cmap="vlag", center=0, 
                                yticklabels=False, cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
    cluster_map.ax_heatmap.set_xlabel("Cell Type")
    cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
    plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=45) 
    
    plt.savefig(out_path + "/tf_activity_compressed_" + condition + ".pdf", bbox_inches='tight')
    plt.close()


    cols_all_ns = tag_mapping.columns[(tag_mapping == "ns").all()]
    tag_mapping = tag_mapping.drop(columns=cols_all_ns)
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.drop(columns=cols_all_ns)

    calc_size = ((len(filtered_summarized_tf_scores_df.columns.unique()) * 1.5), (len(filtered_summarized_tf_scores_df) * 0.2))
    cluster_map = sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "t-score"}, figsize=calc_size, cmap="vlag", center=0, annot= tag_mapping, fmt="", 
                                yticklabels=True, cbar_pos=(1, 0.5, 0.03, 0.05), dendrogram_ratio=(0.2, 0.05))
    cluster_map.ax_heatmap.set_xlabel("Cell Type")
    cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
    plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=45) 

    plt.savefig(out_path + "/tf_activity_" + condition + ".pdf", bbox_inches='tight')
    plt.close()


    

In [9]:
def h_clust(data):
            dist_matrix = pdist(data.T)
            linkage_matrix = linkage(dist_matrix, method = "average")   
            return linkage_matrix

In [10]:
def plot_condition_tf_activities(tf_activity_tables, out_path):
       
   for result_name, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        #print(name_df)
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        #print(name_df)cha

        tag_mapping = name_df[["tf", "tag", "CellType"]]

        tag_mapping = tag_mapping.pivot(index="tf", columns="CellType", values="tag")
        tag_mapping.fillna("ns", inplace=True)

        cols_all_ns = tag_mapping.columns[(tag_mapping == "ns").all()]
        tag_mapping = tag_mapping.drop(columns=cols_all_ns)

    
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 
        name_df_cluster = name_df_cluster.drop(columns=cols_all_ns)
        h_clust_matrix = h_clust(name_df_cluster)


      
        calc_size = ((len(name_df["CellType"].unique()) * 1.5), (len(name_df_cluster) * 0.2))
      
        cluster_map = sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=calc_size, cmap="vlag", center=0, annot= tag_mapping,
                      col_linkage= h_clust_matrix, fmt="", yticklabels=True, cbar_pos=(1, 0.5, 0.03, 0.05), dendrogram_ratio=(0.2, 0.05))
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=45)  

        plt.savefig(out_path + "/" + result_name + "_cluster_condition_activity_difference.pdf", bbox_inches='tight')
        plt.close()

In [11]:
def plot_condition_tf_activities_compressed(tf_activity_tables, out_path):
 
  
    for result_name, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 

        h_clust_matrix = h_clust(name_df_cluster)
        name_df_cluster.reset_index()

        calc_size = ((len(name_df["CellType"].unique()) * 1.5
        ), (len(name_df_cluster) * 0.06))
        cluster_map = sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=calc_size, cmap="vlag", center=0, annot= None,
                       yticklabels=False, col_linkage= h_clust_matrix, fmt="", cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=45) 
        #cluster_map.figure.suptitle(result_name)
        
        
        plt.savefig(out_path + "/" + result_name +  "_cluster_condition_activity_difference_compressed.pdf", bbox_inches='tight')
        plt.close()

In [12]:
def map_t_value(tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    t_values = []  
    anndataobject_markers = anndataobject_markers.set_index("gene")
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(tf_scores_df.index)):
            b = tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     t_values.append(score)

    t_value_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        't_value': t_values
    })

    return t_value_df

In [13]:
def add_entry(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR):
  df = {"source" : source,
      "target" : target,
      "gene_A" : gene_A,
      "gene_B" : gene_B,
      "type_gene_A" : type_gene_A,
      "type_gene_B" : type_gene_B,
      "MeanLR" : MeanLR}

  return df

In [14]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None, organism = "human"):

  if organism == "human":
    ligands = pd.read_csv("ligands_human.csv")
    R2TF = pd.read_csv("rtf_db_human.csv")
  elif organism == "mouse": 
    ligands = pd.read_csv("ligands_mouse.csv")
    R2TF = pd.read_csv("rtf_db_mouse.csv")
  else:
    NameError("Invalid organism to generate CrossTalkeR input!")

  ligands = ligands.drop_duplicates()
  R2TF = R2TF.set_index("tf")

  sorted_regulon = regulon[["tf", "target"]]
  sorted_regulon = sorted_regulon.set_index("tf")

  tf_activities = tf_activities[tf_activities["t_value"] > 0]
  output_list = []
  df_list_l = {}
  df_list_r = {}
  tf_activities.reset_index(drop = True)

  for row in range(len((tf_activities))):
    
    targets = []
    tf_ligands = []
    tf = str(tf_activities["gene"].iloc[row])

    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]
      if isinstance(targets, str):
        targets = [targets]
      
    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]
        if isinstance(receptors, str):
          receptors = [receptors]
          
    else:
        receptors = []


    if len(targets) > 0:
      tf_ligands = np.intersect1d(targets, ligands)
    #print(tf_ligands)

    if organism == "human":
      if len(tf_ligands) > 0:
        existing_entries = set()
        for ligand in tf_ligands:
            expressed = False
            if ligand in gene_expression.index:
              ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")]]
              if (ex_value != 0):
                expressed = True

            if (expressed == True):
              df_list_l = add_entry(source = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                      target = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                      gene_A =tf_activities.iloc[row, tf_activities.columns.get_loc("gene")],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, tf_activities.columns.get_loc("t_value")]
                                                      )
              
              if (ligand, tf) not in existing_entries:
                existing_entries.add((ligand, tf))
                output_list.append(df_list_l)
              

      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = add_entry(source =  tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                target = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                gene_A= receptor,
                                                gene_B= tf_activities.iloc[row, tf_activities.columns.get_loc("gene")],
                                                type_gene_A= "Receptor",
                                                type_gene_B= "Transcription Factor",
                                                MeanLR= tf_activities.iloc[row, tf_activities.columns.get_loc("t_value")]
                                                )
          output_list.append(df_list_r)

      
          
    else: 
      if len(tf_ligands) > 0:
        existing_entries = set()
        for ligand in tf_ligands:
            expressed = False
            if ligand in gene_expression.index:
              ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")]]
              if (ex_value != 0):
                 expressed = True
          
  
              if (expressed == True):
                df_list_l = add_entry(source = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                      target = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                      gene_A =tf_activities.iloc[row, tf_activities.columns.get_loc("gene")],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, tf_activities.columns.get_loc("t_value")]
                                                      )
                
                if (ligand, tf) not in existing_entries:
                  existing_entries.add((ligand, tf))
                  output_list.append(df_list_l)
                  
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = add_entry(source =  tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                target = tf_activities.iloc[row, tf_activities.columns.get_loc("cluster")],
                                                gene_A= receptor,
                                                gene_B= tf_activities.iloc[row, tf_activities.columns.get_loc("gene")],
                                                type_gene_A= "Receptor",
                                                type_gene_B= "Transcription Factor",
                                                MeanLR= tf_activities.iloc[row, tf_activities.columns.get_loc("t_value")]
                                                )
                                                
          output_list.append(df_list_r)
        #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
        #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)
   

  output_df = pd.DataFrame(output_list)
  output_df["gene_A"] = output_df["gene_A"].apply(lambda x: re.sub("_", "+", x))
  output_df["gene_B"] = output_df["gene_B"].apply(lambda x: re.sub("_", "+", x))
  output_df.drop_duplicates(inplace=True)
  output_df.to_csv("tf_l_r_R_data_cond_contr.csv", index=0)


  return output_df

In [15]:
def generate_intracellular_network(tf_activities, gene_expression, outpath, regulon, organism="human"):

    if len(tf_activities.shape) > 0:
        if organism == "human":
            R2TF = pd.read_csv("rtf_db_human.csv").set_index("tf")
        else:
            R2TF = pd.read_csv("rtf_db_mouse.csv").set_index("tf")

    sorted_regulon = regulon[["tf", "target"]].set_index("tf")

    #preextract values
    tf_genes = tf_activities["gene"].values
    tf_celltypes = tf_activities.iloc[:, 2].values
    tf_scores = tf_activities.iloc[:, 3].values

    TFTG_list = []
    RTF_list = []

    tf_activities = tf_activities[tf_activities["t_value"] > 0]
    tf_activities.reset_index(drop = True)
    for row in range(len(tf_activities)):
        tf = str(tf_genes[row])
        celltype = tf_celltypes[row]
        tf_score = tf_scores[row]

        targets = sorted_regulon.loc[tf, "target"] if tf in sorted_regulon.index else []
        #print(targets)
        if tf in R2TF.index:
           receptors = R2TF.loc[tf, "receptor"]
           receptors = [receptors] if isinstance(receptors, str) else receptors
        else:
           receptors = []

        if len(targets) > 0 and len(receptors) > 0:
            for target in targets:
          
                if target in gene_expression.index:
                    ex_value = gene_expression.at[target, celltype]
         
                    if ex_value != 0:
                        TFTG_list.append({
                            "celltype": celltype,
                            "TF": tf,
                            "Target_Gene": target,
                            "TF_Score": tf_score
                        })

            for receptor in receptors:
                RTF_list.append({
                    "TF": tf,
                    "Receptor": receptor
                })

    TFTG_df = pd.DataFrame(TFTG_list)
    RTF_df = pd.DataFrame(RTF_list)

    recept_regulon = pd.merge(RTF_df, TFTG_df, on="TF")
    recept_regulon = recept_regulon.sort_values("TF")
    return recept_regulon

In [16]:
#decoupler condition comparison
def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list, num_cell_filter = 0):

    vs_df_dic = {}

    if isinstance(comparison_list[0], str):  
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = pd.DataFrame()
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts.iloc[0] + metadata_counts.iloc[1]) > num_cell_filter:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
        
                    #finding markers for the condition comparison
                    res_tmp = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="t-test")
                    res_tmp.rename(columns={"group": "condition", "statistic" : "scores"}, inplace=True)
            
                    #calculating wilcoxon scores to use for r value calculation (used in heatmaps)
                    res_heatmap = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="wilcoxon")
                    res_heatmap.rename(columns={"group": "condition", "statistic" : "scores"}, inplace=True)

                    group1 = comparison_sub.X[g == vs1]
                    group2 = comparison_sub.X[g == vs2]
                        
                    res_heatmap["r"] = (res_heatmap["scores"] / np.sqrt(len(group1) + len(group2)))
                    res_heatmap["CellType"] = i
                    res_tmp["CellType"] = i
                    _, res_tmp["FDR"], _, _ = multipletests(res_tmp["pvals"], alpha=0.05, method='fdr_bh')
                    
                    
                    res_tmp = res_tmp[res_tmp["condition"] == vs1]

                    res_heatmap = res_heatmap[["names", "r", "CellType", "condition"]]
                    res_tmp = pd.merge(res_tmp, res_heatmap, on = ["names","CellType", "condition"])

                    res = pd.concat([res, res_tmp], ignore_index=True)

        res_df = res.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df = res_df.assign(tag=res_df["FDR"].apply(assign_significance_tag))

        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
    return vs_df_dic

In [17]:
#decoupler significant tfs

def get_significant_tfs(tf_activities_sub, condition, out_path, tf_condition_significant, celltype, pval, meanchange, plot, condition_comparison = False):
    

    renamed_condition = condition.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    #number_of_clusters = len(tf_activities_sub.obs[celltype].cat.categories) 
    number_of_clusters = (len(tf_activities_sub.obs[celltype].unique()))
                          
    anndataobject_markers_wilcoxon = dc.rank_sources_groups(tf_activities_sub, groupby= celltype, reference="rest", method="t-test")

    anndataobject_markers_wilcoxon.rename(columns={"names" : "gene", "group": "cluster", "statistic" : "scores"}, inplace=True)
    
    
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["meanchange_tag"] = None
    
    #print("after concat", anndataobject_markers_wilcoxon)
    
    anndataobject_markers_wilcoxon["tag"] = anndataobject_markers_wilcoxon["pvals_adj"].apply(eval_pval)
    anndataobject_markers_wilcoxon["meanchange_tag"] = anndataobject_markers_wilcoxon["meanchange"].apply(eval_meanchange_tag)
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_wilcoxon_test.csv",index=0)

   #tag mapping wilcoxon
    clusters = sorted(anndataobject_markers_wilcoxon["cluster"].unique())
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "meanchange_tag", "cluster", "pvals_adj", "meanchange"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval))] 
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["meanchange"] > float(meanchange)) | 
                              (tag_mapping_wilcox["meanchange"] < -float(meanchange))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox = tag_mapping_wilcox[clusters]
    tag_mapping_wilcox = tag_mapping_wilcox.astype("object")
    tag_mapping_wilcox.fillna("ns", inplace=True)

    #tag_mapping_wilcox.to_csv("tag_mapping.csv",index=0)

    tf_activities_sub.obs_names = tf_activities_sub.obs[celltype].astype(str)
    tf_scores_df = tf_activities_sub.to_df()
    #tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    #print(tag_mapping_wilcox)
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num]


    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.sort_index(axis=1, inplace=True)
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f"{single_result_path}/tf_scores_{condition}.csv")
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    filtered_summarized_tf_scores_df.index = filtered_summarized_tf_scores_df.index.map(lambda x: re.sub(".,", "_", x))




    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_t_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged.where(anndataobject_markers_wilcoxon_merged["t_value"] > 0, np.nan) 
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "t_value"]]


    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "t_value"]]

    res_wilcoxon.to_csv(single_result_path + "/significant_cluster_tf_results_wilcoxon_" + renamed_condition + ".csv", index=0)
    #res_t_test.to_csv(single_result_path  + "/_significant_cluster_tf_results_t_test_" + renamed_condition + ".csv", index=0)

    res = {}
    res["cluster"] = res_wilcoxon
    

    if condition_comparison:
        unfiltered_tf_scores = unfiltered_tf_scores.where(unfiltered_tf_scores > 0, np.nan) 
        tf_condition_significant["gene"] = tf_condition_significant["gene"].apply(lambda x: re.sub(".,", "_", x))
        tf_condition_significant = tf_condition_significant.merge(map_t_value(unfiltered_tf_scores, tf_condition_significant), left_on=None, right_on=None, left_index=False, right_index=False)
        tf_condition_significant.dropna(inplace=True)
    
        res["condition"] = tf_condition_significant
        res["condition"].to_csv(f"{single_result_path}/significant_condition_tf_results_{condition}.csv", index=0)

    return res

In [18]:
def IntraTalker_analysis(anndataobject, tf_activities = None, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if arguments_list["decoupler_matrix_format"] == "R":
        anndataobject = anndataobject.T

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"


    condition = anndataobject.obs[arguments_list["condition"]]

    if isinstance(tf_activities, str):
         tf_activities = ad.read_csv(tf_activities)
         tf_activities.obs = anndataobject.obs.reindex(tf_activities.obs.index)
         tf_activities.obsm = anndataobject.obsm
         tf_activities.uns = anndataobject.uns

    elif isinstance(tf_activities, pd.DataFrame):
         tf_activities = ad.AnnData(tf_activities)
         tf_activities.obs = anndataobject.obs.reindex(tf_activities.obs.index)
         tf_activities.obsm = anndataobject.obsm
         tf_activities.uns = anndataobject.uns

    elif tf_activities is None:
         raise NameError("Please attach a csv file with the tf activity values. (For further clarification view the 'Decoupler' section of the vignette.)")
    
    sc.pp.scale(tf_activities)

    #sets the stage for decision if single condition or comparison analysis is done
    
    if not arguments_list["comparison_list"] is None:
        if (len(arguments_list["comparison_list"]) > 0) & (len(pd.unique(anndataobject.obs[arguments_list["condition"]])) < 2):
            arguments_list["comparison_list"] = None
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if arguments_list["comparison_list"] is None:

        result_list = {}
        gene_expression_list = {}
        CTR_cluster_list = {}
        intranet_cluster_list = {}

        #creates loop until after tf activity score
        
        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable].copy()
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable].copy()

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])
        
            
            tf_activity_scores = get_significant_tfs(tf_activities_sub,
                                                        name_iterable,
                                                        tf_path,
                                                        None,
                                                        celltype = arguments_list["celltype"],
                                                        pval = arguments_list["pval"],
                                                        meanchange = arguments_list["meanchange"],
                                                        plot = arguments_list["plot"],
                                                        condition_comparison= False)
            

            result_list[name_iterable] = tf_activity_scores
            
            gene_expression_list[name_iterable] = sub_object_avg
            
        
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
              
            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            

        tf = make_TFOBj(
                tf_activities_condition = list(),
                tf_activities_cluster = result_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = list(),
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = list(),
                intracellular_network_cluster = intranet_cluster_list)

        #with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
        #    pickle.dump(tf, file)

        return tf

    else:
        out_path_compared = (tf_path + "compared")
        if not os.path.isdir(out_path_compared):
            os.mkdir(out_path_compared)

        compared_significant_tfs = condition_comparison_significant(tf_activities, out_path_compared, arguments_list["celltype"], 
                                                                    arguments_list["condition"], arguments_list["comparison_list"], 
                                                                    arguments_list["num_cell_filter"])
        
        print("compared tfs done")
        
        if arguments_list["plot"] == True:
            plot_condition_tf_activities(compared_significant_tfs, out_path_compared)
            plot_condition_tf_activities_compressed(compared_significant_tfs, out_path_compared)

    
        result_condition_list = {}
        result_cluster_list = {}
        gene_expression_list = {}
        CTR_condition_list = {}
        CTR_cluster_list = {}
        intranet_condition_list = {}
        intranet_cluster_list = {}

        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable]
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable]
            

            compared_tfs = pd.DataFrame({"gene" : pd.Series(dtype="str"), "tag" : pd.Series(dtype="str"), "cluster" : pd.Series(dtype="str")})
        
            for result_name, df in compared_significant_tfs.items(): 
                if name_iterable in result_name:
                    tf_condition_significant = compared_significant_tfs[result_name]
                    tf_condition_significant = tf_condition_significant[tf_condition_significant["FDR"] < arguments_list["pval"]]
                    tf_condition_significant = tf_condition_significant[(tf_condition_significant["meanchange"] > float(arguments_list["meanchange"])) | (tf_condition_significant["meanchange"] < (0 - float(arguments_list["meanchange"])))]
                    tf_condition_significant = tf_condition_significant[["tf", "tag", "CellType"]]
                    tf_condition_significant.rename(columns={"tf":"gene", "CellType": "cluster"}, inplace=True)
                    compared_tfs = pd.concat([compared_tfs, tf_condition_significant])

        
            
            re.sub("([,;.:-])", "_", name_iterable)

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], 
                                               outpath= arguments_list["out_path"])
            
            tf_activity_scores = get_significant_tfs(tf_activities_sub,
                                               name_iterable,
                                               tf_path,
                                               compared_tfs,
                                               celltype = arguments_list["celltype"],
                                               pval = arguments_list["pval"],
                                               meanchange = arguments_list["meanchange"],
                                               plot = arguments_list["plot"],
                                               condition_comparison= True)
            
            print("tf_activities done")

            result_condition_list[name_iterable] = tf_activity_scores["condition"]
            result_cluster_list[name_iterable] = tf_activity_scores["cluster"]


            gene_expression_list[name_iterable] = sub_object_avg
            
        
            CTR_condition_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["condition"],
                                                                            gene_expression_list[name_iterable],
                                                                           arguments_list["out_path"],                                             
                                                                           arguments_list["reg"],
                                                                           arguments_list["organism"])
            
            print("CTR input condition done")
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
            print("CTR input cluster done")

            intranet_condition_list[name_iterable] = generate_intracellular_network(tf_activity_scores["condition"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
    
            print("intranet input condition done")

            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            print("intranet input cluster done")
            #print(CTR_cluster_list[name_iterable])
            #print(CTR_condition_list[name_iterable])
            
        tf = make_TFOBj(
                tf_activities_condition = result_condition_list,
                tf_activities_cluster = result_cluster_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = CTR_condition_list,
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = intranet_condition_list,
                intracellular_network_cluster = intranet_cluster_list)

        #with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
        #    pickle.dump(tf, file)
        
        return tf
            

In [192]:
tmpp = sc.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
reg = pd.read_csv("LR2TF_test_run/filterd_regulon.csv")

dc.run_ulm(mat=tmpp, net=reg, source='source', target='target', weight='weight', verbose=True, use_raw=False)

estimates =tmpp.obsm['ulm_estimate']
tmpp.obsm["ulm_estimate"]

1596 features of mat are empty, they will be removed.
Running ulm on mat with 498 samples and 9830 targets for 141 sources.


100%|██████████| 1/1 [00:00<00:00, 13.37it/s]


,AHR,AR,ARNT,ATF1,ATF2,ATF3,ATF4,ATF6,BACH1,BCL6,...,TP73,TWIST1,USF1,USF2,VDR,WT1,YY1,ZBTB33,ZEB1,ZNF263
H18_19086_TB_AAATGCCAGTGTACGG-1,-0.325906,-0.358793,2.993710,1.619173,-0.534671,-0.078113,-0.613460,-0.443634,-0.550418,0.102025,...,-0.336646,-0.288940,1.951161,1.460380,0.387064,0.036205,-1.123608,-0.443631,-0.211403,-1.754107
H18_19086_TB_AACTCTTCATCGTCGG-1,-0.315984,0.421477,3.141227,3.050140,1.410944,-0.075735,-0.594783,-0.430128,0.856580,-3.338824,...,-0.326397,-0.280143,1.787498,-0.653394,0.500689,0.035103,-0.216114,2.216989,1.931203,-0.549862
H18_19086_TB_AATCGGTAGAGAGCTC-1,2.926114,-0.205126,2.748416,-0.569662,-0.572027,-2.438098,1.024219,1.530754,0.655075,-2.966506,...,-0.360167,-0.309127,1.753196,2.721181,0.592756,1.675334,-0.658278,-0.474626,-0.226172,-0.332084
H18_19086_TB_ACACCAAGTAAGTGTA-1,-0.311565,1.244379,-0.292659,-0.509029,-0.511143,-0.074676,-0.586464,-0.424112,-0.526197,0.097536,...,-0.321832,-0.276225,0.632570,1.226090,1.226637,0.034612,-1.175481,-0.424109,-0.202100,-1.086718
H18_19086_TB_ACAGCCGTCATCTGTT-1,-0.298459,-0.519062,-0.280349,-0.487617,-0.489642,2.699297,-0.561795,-0.406272,2.424264,-3.526518,...,-0.308295,-0.264606,0.704175,1.301231,3.129439,0.033156,-1.192200,-0.406269,-0.193599,-1.000435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H18_19892_TB_CAGCAGCGTAACGCGA-1,-0.157385,-0.671958,-0.147835,-0.257130,-0.258198,-0.037722,-0.296245,-0.214237,-0.265802,0.049270,...,-0.162569,-0.139531,-0.412792,-0.325434,-0.674567,0.017483,-0.107640,-0.214234,-0.102089,0.336059
H18_19892_TB_GGTGCGTGTTTGTTTC-1,-0.172655,0.622051,-0.162179,-0.282078,-0.283250,-0.041382,-0.324988,-0.235023,-0.291591,0.054050,...,-0.178343,-0.153069,4.096654,2.576151,0.365094,0.019180,-0.118084,-0.235021,-0.111994,0.161071
H18_19892_TB_TACCTATCACAGGCCT-1,-0.266279,-1.136935,-0.250122,0.872418,-0.436847,-0.063822,1.690018,-0.362467,-0.449712,0.083359,...,-0.275053,-0.236075,3.743020,1.565482,1.580169,0.029580,-0.182117,2.252446,-0.172724,-0.090670
H18_19892_TB_TAGAGCTGTTAGATGA-1,-0.159131,-0.679413,-0.149475,-0.259983,-0.261062,-0.038141,-0.299532,-0.216613,-0.268751,0.049816,...,-0.164373,-0.141079,2.011739,-0.329045,1.679758,0.017677,-0.108834,-0.216612,4.225257,-0.856378


In [37]:
result = IntraTalker_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", 
                              tf_activities= "decoupler_results_test.csv",
                              arguments_list= {"out_path" : "script_test", 
                                                "celltype" : "new_annotation",
                                                "condition" : "protocol", 
                                                "organism" : "human", 
                                                "comparison_list" : ["PMF,MF2", "control"], #["control", "PMF,MF2"]], 
                                                "meanchange" : "0.5",
                                                "pval" : None, 
                                                "num_cell_filter": None,
                                                "reg" : "LR2TF_test_run/filterd_regulon.csv",                                                                                              
                                                "plot" : True,
                                                "decoupler_matrix_format" : "Python"})


vs1: PMF,MF2, vs2: control
compared tfs done
tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done
tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done


In [3]:
tmpp = sc.read_h5ad("Il1rn_KO_Maus_Data/Myeloid_Stromal_LSK_integrated_harmony.h5ad")
reg = pd.read_csv("Il1rn_KO_Maus_Data/decoupler_mousereg_AB.csv")

dc.run_ulm(mat=tmpp, net=reg, source='source', target='target', weight='weight', verbose=True, use_raw=False)

estimates = tmpp.obsm['ulm_estimate']
estimates.to_csv("Il1rn_KO_Maus_Data/decoupler_results_mouse.csv")


7351 features of mat are empty, they will be removed.
Running ulm on mat with 35435 samples and 20647 targets for 160 sources.


100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


In [19]:
mouse_result = IntraTalker_analysis(anndataobject= "Il1rn_KO_Maus_Data/Myeloid_Stromal_LSK_integrated_harmony.h5ad", 
                                    tf_activities= "Il1rn_KO_Maus_Data/decoupler_results_mouse.csv",   
                                    arguments_list= {"out_path": "Il1rn_KO_Maus_Data/test",
                                                    "reg" : "Il1rn_KO_Maus_Data/decoupler_mousereg_AB.csv",
                                                    "organism" : "mouse",
                                                    "celltype" : "cluster_names",
                                                    "condition" : "Condition",
                                                    "comparison_list": ["KO", "WT"],
                                                    "meanchange" : 0.4,
                                                    "pval" : 0.01,
                                                    "num_cell_filter" : None,
                                                    "plot" : True,
                                                    "decoupler_matrix_format" : "Python"})
                            

vs1: KO, vs2: WT
compared tfs done
tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done
tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done


In [194]:
#inputt = pd.read_csv("new_test//TF_results//control//significant_condition_tf_results_control.csv")
inputt = pd.read_csv("new_test/TF_results/PMF_MF2/significant_condition_tf_results_PMF_MF2.csv")
inputt = inputt.iloc[:,1:5]
inputt.rename(columns={"z_score" : "t_value"}, inplace=True)
#inputt = inputt.set_index("tf")
#print(inputt)

reguloninput = pd.read_csv("filterd_regulon.csv")
reguloninput.rename(columns={"source" : "tf"}, inplace=True)

generate_CrossTalkeR_input(inputt, result.average_gene_expression["control"], "R_CTR_input_w_py_code", regulon = reguloninput, organism = "human")

,source,target,gene_A,gene_B,type_gene_A,type_gene_B,MeanLR
0,Megakaryocyte,Megakaryocyte,ATF3,ANXA1,Transcription Factor,Ligand,0.190079
1,Megakaryocyte,Megakaryocyte,ATF3,ANXA2,Transcription Factor,Ligand,0.190079
2,Megakaryocyte,Megakaryocyte,ATF3,CCL5,Transcription Factor,Ligand,0.190079
3,Megakaryocyte,Megakaryocyte,ATF3,CIRBP,Transcription Factor,Ligand,0.190079
4,Megakaryocyte,Megakaryocyte,ATF3,SORL1,Transcription Factor,Ligand,0.190079
...,...,...,...,...,...,...,...
5280,Neural,Neural,INSR,ZNF24,Receptor,Transcription Factor,0.532363
5281,Neural,Neural,NGFR,ZNF274,Receptor,Transcription Factor,1.279600
5282,Neural,Neural,ZNF318,LGALS1,Transcription Factor,Ligand,0.713510
5283,Neural,Neural,ZNF589,ANP32B,Transcription Factor,Ligand,0.506688


In [195]:
result.CTR_input_condition["control"].to_csv("py_ctr_input_wo_ctr_exp_tables.csv", index = False)
result.average_gene_expression


{'control': new_annotation  Neural       MSC  Fibroblast  Megakaryocyte   Myeloid
 FO538757.2         0.0  1.838987    0.000000       0.000000  0.000000
 RP11-206L10.9      0.0  0.000000    0.386757       0.000000  0.000000
 SAMD11             0.0  0.000000    0.000000       0.000000  0.000000
 NOC2L              0.0  0.197924    2.134077       0.780153  0.561325
 HES4               0.0  0.545435    0.000000       1.190760  0.000000
 ...                ...       ...         ...            ...       ...
 SMYD5              0.0  0.000000    0.000000       0.000000  0.000000
 CCDC85C            0.0  0.000000    0.000000       0.000000  0.000000
 CPNE2              0.0  0.000000    0.000000       0.000000  0.000000
 TMEM206            0.0  0.000000    0.000000       0.000000  0.000000
 CCDC9              0.0  0.000000    0.000000       0.000000  0.000000
 
 [11426 rows x 5 columns],
 'PMF,MF2': new_annotation    Neural       MSC  Fibroblast  Megakaryocyte   Myeloid
 FO538757.2      0.64295

In [196]:
result.intracellular_network_condition["control"].to_csv("py_intra_network_ctrl.csv")
result.intracellular_network_condition["PMF,MF2"].to_csv("py_intra_network_PMF.csv")

In [197]:
result.intracellular_network_cluster["control"].to_csv("py_intra_network_ctrl_cluster.csv")
result.intracellular_network_cluster["PMF,MF2"].to_csv("py_intra_network_PMF_cluster.csv")

In [198]:
def add_node_type(df):
    
    df['gene_A'] = np.where(df['type_gene_A'] == 'Ligand', df['gene_A'] + '|L', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Receptor', df['gene_A'] + '|R', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Transcription Factor', df['gene_A'] + '|TF', df['gene_A'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Ligand', df['gene_B'] + '|L', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Receptor', df['gene_B'] + '|R', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Transcription Factor', df['gene_B'] + '|TF', df['gene_B'])
    return df


In [199]:
def combine_LR_and_TF(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    lr_table = pd.read_csv(LR_prediction, index_col=0)
  

  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])
    #print(condition, celltype, lr_receptors)

    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(lr_receptors)]
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]


    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, tf_ligand_interactions])
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
    
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [200]:
def combine_LR_and_TF_unfiltered(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    lr_table = pd.read_csv(LR_prediction, index_col=0)
  


  complete_interactions = pd.concat([tf_table, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + "_unfiltered.csv"), index=0)
  return(complete_interactions)

In [ ]:
def combine_LR_and_TF_complexes(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    lr_table = pd.read_csv(LR_prediction, index_col=False)
  
  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])
    #print(lr_filtered_receptors)

    lr_receptors = pd.Series(lr_receptors)
    contains_complex = lr_receptors.str.contains("_", na=False)
    
    R_with_complex = lr_receptors[contains_complex]
    #print("R_with_complex", R_with_complex)
    R_without_complex = lr_receptors[(~contains_complex)]
  
    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(R_without_complex)]

    c_receptors = tf_table_receptors[tf_table_receptors["gene_A"].apply(lambda x: any(gene in x.split("+") for gene in lr_receptors))]

    complex_df = pd.DataFrame()
    if len(R_with_complex) > 0:
      for complex in R_with_complex:
        receptors = complex.split("_")
        R_TF_with_complex = tf_table_receptors[tf_table_receptors["gene_A"].isin(receptors)]
 
        if len(R_TF_with_complex) == 0:
          continue
        
        R_TF_with_complex.drop_duplicates()
        R_TF_with_complex.loc[:,"gene_A"] = complex
        complex_df = pd.concat([complex_df, R_TF_with_complex])

      #complex_df.drop_duplicates()

    tf_receptor_interactions = pd.concat([tf_receptor_interactions, complex_df])
    #print("tf_receptor_interactions", tf_receptor_interactions)
    
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]

    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, c_receptors, tf_ligand_interactions])
  
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
  #print("intra_connections", intra_connections)
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])
  
  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=False) #quoting= csv.QUOTE_NONNUMERIC)
  return(complete_interactions)


In [202]:
#WINDOWS
table_ctr = pd.read_csv("LR2TF_test_run\\CTR_LR.csv" )
table_exp = pd.read_csv("LR2TF_test_run\\EXP_LR.csv")
table_ctr.drop('Unnamed: 0', axis=1, inplace=True)
table_exp.drop('Unnamed: 0', axis=1, inplace=True)


ctr_input = combine_LR_and_TF_complexes(result.CTR_input_condition["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF_complexes(result.CTR_input_condition["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")


ctr_input_cluster = combine_LR_and_TF_complexes(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control_cluster")
exp_input_cluster = combine_LR_and_TF_complexes(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2_cluster")

In [38]:
#LINUX
table_ctr = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/CTR_LR.csv", index_col=False)
table_exp = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/EXP_LR.csv", index_col= False)
table_ctr.drop('Unnamed: 0', axis=1, inplace=True)
table_exp.drop('Unnamed: 0', axis=1, inplace=True)

ctr_input = combine_LR_and_TF_complexes(result.CTR_input_condition["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF_complexes(result.CTR_input_condition["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")

ctr_input_cluster = combine_LR_and_TF_complexes(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control_cluster")
exp_input_cluster = combine_LR_and_TF_complexes(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2_cluster")

In [47]:
#LINUX MOUSE
table_ctr = pd.read_csv("Il1rn_KO_Maus_Data/WT_lr_ready.csv", index_col=False)
table_exp = pd.read_csv("Il1rn_KO_Maus_Data/KO_lr_ready.csv", index_col= False)
table_ctr.drop('Unnamed: 0', axis=1, inplace=True)
table_exp.drop('Unnamed: 0', axis=1, inplace=True)

ctr_input = combine_LR_and_TF_complexes(mouse_result.CTR_input_condition["WT"], table_ctr, "Il1rn_KO_Maus_Data/test/", "WT")
exp_input = combine_LR_and_TF_complexes(mouse_result.CTR_input_condition["KO"], table_exp, "Il1rn_KO_Maus_Data/test/", "KO")

ctr_input_cluster = combine_LR_and_TF_complexes(mouse_result.CTR_input_cluster["WT"], table_ctr, "Il1rn_KO_Maus_Data/test/", "WT_cluster")
exp_input_cluster = combine_LR_and_TF_complexes(mouse_result.CTR_input_cluster["KO"], table_exp, "Il1rn_KO_Maus_Data/test/", "KO_cluster")
